In [6]:
import os
import random
from data import prepare_paths, get_dataset
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageOps
from keras.utils import load_img
import tensorflow as tf
import keras

# Ensure dataset folder exists
os.makedirs("dataset", exist_ok=True)

# Download and extract the Oxford Pets dataset
!wget -P dataset https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -P dataset https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

!tar -xf dataset/images.tar.gz -C dataset
!tar -xf dataset/annotations.tar.gz -C dataset

print("Dataset downloaded and extracted successfully.")

# Set paths and parameters
input_dir = "dataset/images/"
target_dir = "dataset/annotations/trimaps/"
img_size = (160, 160)
batch_size = 16
num_classes = 3

# Prepare input and target image paths
input_img_paths, target_img_paths = prepare_paths(input_dir, target_dir)

# Debug: Display number of samples
print(f"Number of input images: {len(input_img_paths)}")
print(f"Number of target masks: {len(target_img_paths)}")

# Split into training and validation sets
val_samples = 1000
random.seed(1337)
random.shuffle(input_img_paths)
random.shuffle(target_img_paths)

train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

print("Training samples:", len(train_input_img_paths))
print("Validation samples:", len(val_input_img_paths))

# Set max_dataset_len to a smaller value for faster training
max_dataset_len = 1000  # Use 1000 samples for debugging; set to None for full dataset

# Create training and validation datasets
train_dataset = get_dataset(batch_size, img_size, train_input_img_paths, train_target_img_paths, max_dataset_len)
valid_dataset = get_dataset(batch_size, img_size, val_input_img_paths, val_target_img_paths, max_dataset_len)

print("Datasets prepared successfully.")

def get_unet_model(img_size, num_classes):
    inputs = layers.Input(shape=img_size + (3,))

    # Downsampling
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
        x = layers.add([x, residual])
        previous_block_activation = x

    # Upsampling
    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])
        previous_block_activation = x

    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    return Model(inputs, outputs)

# Instantiate model
model = get_unet_model(img_size, num_classes)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# Define callbacks
callbacks = [ModelCheckpoint("oxford_segmentation.keras", save_best_only=True)]

# Model fit
model.fit(
    train_dataset,
    epochs=2,
    validation_data=valid_dataset,
    callbacks=callbacks
)


--2024-12-22 08:30:46--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2024-12-22 08:30:46--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘dataset/images.tar.gz.5’

images.tar.gz.5     100%[===================>] 755.23M  33.0MB/s    in 24s     

2024-12-22 08:31:11 (31.2 MB/s) - ‘dataset/images.tar.gz.5’ saved [791918971/791918971]

--2024-12-22 08:31:11--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.ta

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 160, 160, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 80, 80, 32)     │            896 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_75    │ (None, 80, 80, 32)     │            128 │ conv2d_45[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_75             │ (None, 80, 80, 32)     │              0 │ batch_normalization_7… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_76             │ (None, 80, 80, 32)     │              0 │ activation_75[0][0]    │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_30       │ (None, 80, 80, 64)     │          2,400 │ activation_76[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_76    │ (None, 80, 80, 64)     │            256 │ separable_conv2d_30[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_77             │ (None, 80, 80, 64)     │              0 │ batch_normalization_7… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_31       │ (None, 80, 80, 64)     │          4,736 │ activation_77[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_77    │ (None, 80, 80, 64)     │            256 │ separable_conv2d_31[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 40, 40, 64)     │              0 │ batch_normalization_7… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 40, 40, 64)     │          2,112 │ activation_75[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_35 (Add)              │ (None, 40, 40, 64)     │              0 │ max_pooling2d_15[0][0… │
│                           │                        │                │ conv2d_46[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_78             │ (None, 40, 40, 64)     │              0 │ add_35[0][0]           │
│ (Activation)         

 Total params: 2,058,979 (7.85 MB)

 Trainable params: 2,055,203 (7.84 MB)

 Non-trainable params: 3,776 (14.75 KB)

Epoch 1/2
     63/Unknown 303s 5s/step - accuracy: 0.5602 - loss: 1.7199

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


63/63 ━━━━━━━━━━━━━━━━━━━━ 391s 6s/step - accuracy: 0.5607 - loss: 1.7142 - val_accuracy: 0.5643 - val_loss: 0.9925
Epoch 2/2
63/63 ━━━━━━━━━━━━━━━━━━━━ 361s 6s/step - accuracy: 0.6296 - loss: 0.9844 - val_accuracy: 0.5762 - val_loss: 1.1960


# Imports and Setup

In [15]:
# Imports
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageOps
from keras.utils import load_img
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from data import prepare_paths, get_dataset

# Ensure dataset folder exists
os.makedirs("dataset", exist_ok=True)

# Global parameters
input_dir = "dataset/images/"
target_dir = "dataset/annotations/trimaps/"
img_size = (160, 160)
batch_size = 32
num_classes = 3
val_samples = 1000


# Download and Extract Dataset

In [16]:
# Download and extract the Oxford Pets dataset
!wget -P dataset https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -P dataset https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

!tar -xf dataset/images.tar.gz -C dataset
!tar -xf dataset/annotations.tar.gz -C dataset

print("Dataset downloaded and extracted successfully.")


--2024-12-22 09:13:00--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2024-12-22 09:13:01--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘dataset/images.tar.gz.7’

images.tar.gz.7     100%[===================>] 755.23M  33.6MB/s    in 24s     

2024-12-22 09:13:25 (31.1 MB/s) - ‘dataset/images.tar.gz.7’ saved [791918971/791918971]

--2024-12-22 09:13:26--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.ta

# Prepare Dataset Paths


In [17]:
# Prepare input and target image paths
input_img_paths, target_img_paths = prepare_paths(input_dir, target_dir)

# Debug: Display number of samples
print(f"Number of input images: {len(input_img_paths)}")
print(f"Number of target masks: {len(target_img_paths)}")


Number of input images: 7390
Number of target masks: 7390


# Split Dataset

In [22]:
# Split into training and validation sets
random.seed(1337)
random.shuffle(input_img_paths)
random.shuffle(target_img_paths)

train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

print("Training samples:", len(train_input_img_paths))
print("Validation samples:", len(val_input_img_paths))

max_dataset_len = 1000  # Use 1000 samples for debugging; set to None for full dataset

# Create training and validation datasets
train_dataset = get_dataset(batch_size, img_size, train_input_img_paths, train_target_img_paths,max_dataset_len, augment=True)
valid_dataset = get_dataset(batch_size, img_size, val_input_img_paths, val_target_img_paths, max_dataset_len, augment=True)

print("Datasets prepared successfully.")


Training samples: 6390
Validation samples: 1000


TypeError: get_dataset() got an unexpected keyword argument 'augment'

# Define the U-Net Model

In [13]:
def get_unet_model(img_size, num_classes):
    inputs = layers.Input(shape=img_size + (3,))

    # Downsampling
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
        x = layers.add([x, residual])
        previous_block_activation = x

    # Upsampling
    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])
        previous_block_activation = x

    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    return Model(inputs, outputs)


# Compile and Train the Model

In [14]:
# Instantiate and compile the model
model = get_unet_model(img_size, num_classes)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# Define callbacks
callbacks = [ModelCheckpoint("oxford_segmentation.keras", save_best_only=True)]

# Train the model
history = model.fit(
    train_dataset,
    epochs=1,
    validation_data=valid_dataset,
    callbacks=callbacks
)

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 160, 160, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_54 (Conv2D)        │ (None, 80, 80, 32)     │            896 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_90    │ (None, 80, 80, 32)     │            128 │ conv2d_54[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_90             │ (None, 80, 80, 32)     │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_91             │ (None, 80, 80, 32)     │              0 │ activation_90[0][0]    │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_36       │ (None, 80, 80, 64)     │          2,400 │ activation_91[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_91    │ (None, 80, 80, 64)     │            256 │ separable_conv2d_36[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_92             │ (None, 80, 80, 64)     │              0 │ batch_normalization_9… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_37       │ (None, 80, 80, 64)     │          4,736 │ activation_92[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_92    │ (None, 80, 80, 64)     │            256 │ separable_conv2d_37[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_18          │ (None, 40, 40, 64)     │              0 │ batch_normalization_9… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_55 (Conv2D)        │ (None, 40, 40, 64)     │          2,112 │ activation_90[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_42 (Add)              │ (None, 40, 40, 64)     │              0 │ max_pooling2d_18[0][0… │
│                           │                        │                │ conv2d_55[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_93             │ (None, 40, 40, 64)     │              0 │ add_42[0][0]           │
│ (Activation)         

 Total params: 2,058,979 (7.85 MB)

 Trainable params: 2,055,203 (7.84 MB)

 Non-trainable params: 3,776 (14.75 KB)

     32/Unknown 314s 9s/step - accuracy: 0.5464 - loss: 1.9816

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


32/32 ━━━━━━━━━━━━━━━━━━━━ 416s 13s/step - accuracy: 0.5473 - loss: 1.9699 - val_accuracy: 0.4111 - val_loss: 1.5983
